# Find the best sample position based on a fit of the total diffraction intensity

Use the total diffracted signal on the pilatus detector to determine the optimal motor position for a given motor scan. 

In [ ]:
%matplotlib widget
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

In [ ]:
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/scan-XXXX.h5'
fname = DM.getLatestScan(scan_type='dscan')

# get the motor name from the scan type
motor = DM.getScanType(fname).split()[1]
meta = DM.getMetaData(fname,custom_keys={'motor':f'/entry/instrument/{motor}/value'})
m = meta['motor']

# read data form the .h5 file iteratively to 
# ensure compatibility with large datasets
dset = '/entry/instrument/pilatus/data'
with h5.File(fname,'r') as f:
    I = np.zeros(f[dset].shape[0])
    for i in range(I.shape[0]):
        im = f[dset][i].astype(float)
        im[im<0]=np.nan
        I[i] = np.nanmean(im)

# [0-1] normalize
I = (I-I.min())/(I.max()-I.min())

# performe Gaussian single-peak fit
amp,pos,fwhm,bgr,y_calc = DM.fitting.singlePeakFit(m,I)
_m = np.linspace(m[0],m[-1],100)
y_calc = DM.fitting.gauss(_m,amp,pos,fwhm/(2*np.sqrt(2*np.log(2))),bgr)

print(f'Fitted maxima at {motor}: {pos:.4f} mm')
print(f'Closes measured  {motor}: {m[np.argmin(np.abs(m-pos))]:.4f} mm')

# plot the observed and fitted points
plt.figure()
plt.title(DM.getScan_id(fname))
plt.plot(m,I,'x',c='grey',ms=6,label='Mean intensity')
plt.plot(_m,y_calc,'r-',lw=1,label='Gaussian fit')
plt.axvline(pos,color='grey',linestyle=':',lw=2)
plt.annotate(f'{pos:.3f}',(pos,0.5),color='r',fontsize=50,va='top',ha='center')
plt.xlabel(f'{motor} position (mm)')
plt.ylabel('Intensity')
plt.legend()

# ROI alignment
Use a region of interest signal from the integrated data to determine the optimal motor position for a given motor scan.

In [ ]:
# ROI    label  :    roi
rois = {'ROI_1' : [8.5,8.8]}

fname = DM.getLatestScan(scan_type='dscan')

# get the motor name from the scan type
motor = DM.getScanType(fname).split()[1]
meta = DM.getMetaData(fname,custom_keys={'motor':f'/entry/instrument/{motor}/value'})
m = meta['motor']

# get the azimuthally integrated filename from master file name
aname = DM.getAzintFname(fname)

# read the integrated data
data = DM.getAzintData(aname)
if type(data['q']) != type(None):
    x = data['q']
    xlabel = 'Q (A-1)'
else:
    x = data['tth']
    xlabel = '2theta (deg)'
I = data['I']
y_avg = np.mean(I,axis=0)

# initiate plot
fig, [ax0,ax1] = plt.subplots(2)
fig.suptitle(DM.getScan_id(fname))
ax0.set_title('Region of interest')
ax0.set_title('Integrated intensity')
ax0.set_xlabel(xlabel)
ax0.set_xlabel(f'{motor} position (mm)')

# loop over ROIs
for key in rois:
    roi = rois[key]
    roi = (x>roi[0]) & (x<roi[1])
    
    # ROI plot
    ax0.plot(x,y_avg,'grey',label='average')
    ax0.plot(x[roi],y_avg[roi],'.',label=key)
    
    # calculate integral
    signal = np.trapz(I[:,roi],x[roi],axis=1)
    # performe Gaussian single-peak fit
    amp,pos,fwhm,bgr,y_calc = DM.fitting.singlePeakFit(m,signal)
    
    # Integral plot
    ax1.plot(m,signal,label=key)
    ax1.plot(m,y_calc,'r-',lw=1,label='Gaussian fit')
    plt.axvline(pos,color='grey',linestyle=':',lw=1)
    plt.annotate(f'{pos:.3f}',(pos,1),color='r')
    
    print(f'{key}:')
    print(f'Fitted maxima at {motor}: {pos:.4f} mm')
    print(f'Closes measured  {motor}: {m[np.argmin(np.abs(m-pos))]:.4f} mm')
    print()
plt.legend()

# Multi peak alignment
Ensure that it is the correct scan, provide the number of peaks (capillaries) and the desired scan command (timescan or similar)  
Confirm that the fitted positions are correct, then save the printed commands to a text file  
Run the sequence file in `SPOCK` with the `text_sequence` macro. Make sure to use the full sequence file path.  

In [ ]:
#############################################################################

fname = DM.findScan()
num_of_peaks = 3
scan_command = 'timescan 1 1'

#############################################################################

# get the motor name from the scan type
motor = DM.getScanType(fname).split()[1]
meta = DM.getMetaData(fname,custom_keys={'motor':f'/entry/instrument/{motor}/value'})
m = meta['motor']

# read data form the .h5 file iteratively to 
# ensure compatibility with large datasets
dset = '/entry/instrument/pilatus/data'
with h5.File(fname,'r') as f:
    I = np.zeros(f[dset].shape[0])
    for i in range(I.shape[0]):
        im = f[dset][i].astype(float)
        im[im<0]=np.nan
        I[i] = np.nanmean(im)

# [0-1] normalize
I = (I-I.min())/(I.max()-I.min())

# guess regions of interest, assuming equidistant peaks in a symmetric interval
rois = np.linspace(0,m.shape[0],num_of_peaks+1,dtype=int)
x0s = [m[np.argmax(I[rois[i]:rois[i+1]])+rois[i]] for i in range(num_of_peaks)]
# performe Gaussian multi-peak fit
peaks, bgr = DM.fitting.multiPeakFit(m,I,num_of_peaks,x0s=x0s)

# interpolate additional x-values for the y_calc plots 
_m = np.linspace(m.min(),m.max(),m.shape[0]*num_of_peaks)

# plot the observed and fitted points
plt.figure()
plt.title(DM.getScan_id(fname))
plt.xlabel(f'{motor} position (mm)')
plt.ylabel('Intensity')
plt.legend(ncols=2)
plt.plot(m,I,'.-',c='w',ms=3,lw=0.5,label='Mean intensity')
print('Sequence commands:\n')
for peak in peaks:
    a ,x0 , sigma  = peaks[peak]
    y_calc = DM.fitting.gauss(_m,a,x0,sigma,bgr)
    p, = plt.plot(_m,y_calc,'-',lw=1,label=peak)
    c = p.get_color()
    plt.axvline(x0,color=c,linestyle=':',lw=1)
    plt.annotate(f'{x0:.3f}',(x0,a),color=c)
    print(f'umv {motor} {x0:.2f}')
    print(scan_command)
print('movepos ex_situ_cen')
